# Import Modules

In [1]:
#%qtconsole
#%matplotlib inline

import matplotlib.pyplot as plt
from functools import partial
import numpy as np
import time 
import qcodes as qc
import os
from qcodes.math_utils import FieldVector
from qcodes import Station, load_or_create_experiment, initialise_database, Measurement, load_by_run_spec, load_by_guid, initialise_or_create_database_at 
from qcodes.utils.dataset.doNd import do1d, do2d, plot

from qcodes_contrib_drivers.drivers.Tektronix.Keithley_2700 import Keithley_2700
from qcodes.instrument_drivers.tektronix.Keithley_2600_channels import Keithley_2600
from qcodes_contrib_drivers.drivers.Oxford.IPS120 import OxfordInstruments_IPS120
from qcodes.instrument.visa import VisaInstrument
from qcodes_contrib_drivers.drivers.StanfordResearchSystems.SIM928 import SIM928
from qcodes.instrument_drivers.stanford_research.SR830 import SR830
from qcodes.instrument_drivers.stanford_research.SR860 import SR860
from qcodes.dataset.plotting import plot_dataset
from qcodes.instrument_drivers.oxford.MercuryiPS_VISA import MercuryiPS
import IPython.lib.backgroundjobs as bg
from plottr.apps import inspectr

# Connect to Instruments and Initialize Variables

In [2]:
#Magnet Controller
mag120 = MercuryiPS('mips', 'TCPIP0::169.254.255.2::7020::SOCKET')
#Lockins(SR860 and SR830)
rxy1=SR830('Dilbert','GPIB1::6::INSTR' )
rxy2=SR860('Laughlin', 'GPIB1::4::INSTR')
rxy3=SR860('Kiwi','GPIB1::9::INSTR' )
rxy4=SR860('Pangolin', 'GPIB1::12::INSTR')
rxy5=SR830('Wally','GPIB1::1::INSTR' )

Connected to: OXFORD INSTRUMENTS MERCURY IPS (serial:190250049, firmware:2.5.09.000) in 0.19s
Connected to: Stanford_Research_Systems SR830 (serial:s/n48257, firmware:ver1.07) in 0.13s
Connected to: Stanford_Research_Systems SR860 (serial:004479, firmware:V1.51) in 0.04s


C:\Anaconda3\envs\MattMann\lib\site-packages\qcodes\instrument\instrument_base.py:147: QCoDeSDeprecationWarning: Parameter ch1_databuffer on instrument Dilbert does not correctly pass kwargs to its baseclass. A Parameter class must take `**kwargs` and forward them to its baseclass.
  QCoDeSDeprecationWarning,
C:\Anaconda3\envs\MattMann\lib\site-packages\qcodes\instrument\instrument_base.py:147: QCoDeSDeprecationWarning: Parameter ch2_databuffer on instrument Dilbert does not correctly pass kwargs to its baseclass. A Parameter class must take `**kwargs` and forward them to its baseclass.
  QCoDeSDeprecationWarning,


Connected to: Stanford_Research_Systems SR860 (serial:003172, firmware:V1.47) in 0.05s
Connected to: Stanford_Research_Systems SR860 (serial:004478, firmware:V1.51) in 0.04s
Connected to: Stanford_Research_Systems SR830 (serial:s/n48240, firmware:ver1.07) in 0.09s


C:\Anaconda3\envs\MattMann\lib\site-packages\qcodes\instrument\instrument_base.py:147: QCoDeSDeprecationWarning: Parameter ch1_databuffer on instrument Wally does not correctly pass kwargs to its baseclass. A Parameter class must take `**kwargs` and forward them to its baseclass.
  QCoDeSDeprecationWarning,
C:\Anaconda3\envs\MattMann\lib\site-packages\qcodes\instrument\instrument_base.py:147: QCoDeSDeprecationWarning: Parameter ch2_databuffer on instrument Wally does not correctly pass kwargs to its baseclass. A Parameter class must take `**kwargs` and forward them to its baseclass.
  QCoDeSDeprecationWarning,


# Global Parameters for Instruments and Experiment

In [3]:
csvPath = "C:\\Users\\Admin\\Desktop\\MattMann\\8-3-22.1MM11A3\\MicroOhmicResistance3\\MicroOhmicResistance_34,42,37,43,&36_100pA_SweepUpTonu=1.csv"
sample = "8-3-22.1MM11A3"
exp_name = "MicroOhmicResistance_34,42,37,43,&36_100pA_SweepUpTonu=1"
meas_name = 'MicroOhmicResistance_vs_Mag Field'
amp = 0.0000000001
#Sweep rate in T/s
#IMPORTANT: Never go above 50mT/min(0.83mT/s)
sweepRate = 0.00083
fieldTarget = 7.91

In [5]:
dataBasePath = 'C:\\Users\\Admin\Desktop\\MattMann\\8-3-22.1MM11A3\\MicroOhmicResistance3\\MicroOhmicResistance_Vs_MagField_092923.db'

# Set Up Database for Data Collection

In [6]:
db_file_path = os.path.join(os.getcwd(), dataBasePath)
initialise_or_create_database_at(db_file_path)

# Set Magnetic Field Sweep Parameters

In [7]:
#Prints old sweep rate, sets new one, prints new sweep rate
field = mag120._get_ramp_rate()
print("Previous SweepRate")
print(field.z)
rate = FieldVector(0.0,0.0,sweepRate)
mag120._set_ramp_rate(rate)
print("New SweepRate")
print(mag120._get_ramp_rate().z)

Previous SweepRate
0.0008299999999999999
New SweepRate
0.0008299999999999999


In [8]:
#Prints old target field, sets new one, prints new target field
target = FieldVector(0.0,0.0,fieldTarget)
print("Previous Target Field")
print(mag120._get_target_field().z)
mag120._set_target_field(target)
print("New Target Field")
print(mag120._get_target_field().z)

Previous Target Field
3.96
New Target Field
7.91


# Set Up Live Plot

In [9]:
#Opens windows that will plot results as they are collected
jobs = bg.BackgroundJobManager()
jobs.new(inspectr.main, db_file_path)

<BackgroundJob #0: <function main at 0x000000001D7300D8>>

# Setup Station and Measurement Parameters

In [10]:
#Handles equipment
station = qc.Station()
station.add_component(mag120)
station.add_component(rxy1)
station.add_component(rxy2)
station.add_component(rxy3)
station.add_component(rxy4)
station.add_component(rxy5)

#Manages independent and dependent variables
cutoff_freq_exp = load_or_create_experiment(
    experiment_name=exp_name,
    sample_name=sample
)
context_meas = Measurement(exp=cutoff_freq_exp, station=station, name=meas_name)
context_meas.write_period = 0.1

context_meas.register_parameter(mag120.field_measured)
context_meas.register_parameter(rxy1.X, setpoints=(mag120.field_measured,))
context_meas.register_parameter(rxy2.X, setpoints=(mag120.field_measured,))
context_meas.register_parameter(rxy3.X, setpoints=(mag120.field_measured,))
context_meas.register_parameter(rxy4.X, setpoints=(mag120.field_measured,))
context_meas.register_parameter(rxy5.X, setpoints=(mag120.field_measured,))

# Sweep Magnetic Field and Measure Results for MicroOhmics

In [11]:
mag120.ramp('simul')
with context_meas.run() as datasaver:
    while(mag120._get_field().z<7.9):
        set_m = mag120.field_measured.get().z
        get_x1 = rxy1.X.get()/amp
        get_x2 = rxy2.X.get()/amp
        get_x3 = rxy3.X.get()/amp
        get_x4 = rxy4.X.get()/amp
        get_x5 = rxy5.X.get()/amp

        datasaver.add_result((mag120.field_measured, set_m),
                             (rxy1.X, get_x1),
                             (rxy2.X, get_x2),
                             (rxy3.X, get_x3),
                             (rxy4.X, get_x4),
                             (rxy5.X, get_x5))        
        print(set_m)
        time.sleep(1)
    dataset = datasaver.dataset

Starting experimental run with id: 1. 
3.96
3.9601
3.9607
3.9618
3.9626
3.9637
3.9649
3.9658
3.9669
3.9677
3.9689
3.97
3.9715
3.9723
3.9735
3.9744
3.9755
3.9766
3.9774
3.9786
3.9795
3.9807
3.9818
3.9826
3.9838
3.9846
3.9858
3.987
3.9878
3.9889
3.9898
3.9909
3.9921
3.9927
3.9942
3.9953
3.9961
3.9973
3.9982
3.9993
4.0005
4.0012
4.0024
4.0034
4.0045
4.0057
4.0065
4.0076
4.0079
4.0096
4.0107
4.0115
4.0127
4.0136
4.0148
4.0159
4.0167
4.0176
4.0187
4.0199
4.0207
4.0217
4.0228
4.0237
4.0249
4.026
4.0268
4.028
4.0288
4.03
4.0306
4.0321
4.0333
4.0341
4.0353
4.0362
4.0373
4.0384
4.0393
4.0403
4.0412
4.0423
4.0435
4.0443
4.0455
4.0464
4.0475
4.0486
4.0495
4.0506
4.0517
4.0527
4.0538
4.0541
4.0556
4.0568
4.0576
4.0582
4.0596
4.0608
4.0619
4.0628
4.0639
4.0647
4.0659
4.067
4.0679
4.0691
4.0699
4.0711
4.0722
4.0731
4.0743
4.0751
4.0763
4.0773
4.0784
4.0794
4.0805
4.0808
4.0824
4.0835
4.0844
4.0855
4.0864
4.0875
4.0887
4.0895
4.0906
4.0915
4.0927
4.0939
4.0947
4.0959
4.0967
4.0976
4.0987
4.0998
4.100

5.1771
5.1779
5.1789
5.1801
5.1811
5.182
5.1832
5.1842
5.1851
5.1863
5.1872
5.1884
5.1893
5.1903
5.1914
5.1924
5.1933
5.1945
5.1955
5.1964
5.1977
5.1986
5.1995
5.2008
5.2018
5.2026
5.2039
5.2048
5.2057
5.207
5.2079
5.2091
5.21
5.2109
5.2122
5.2131
5.214
5.2153
5.2162
5.2172
5.2184
5.2193
5.2202
5.2215
5.2224
5.2234
5.2246
5.2256
5.2265
5.2277
5.2286
5.2296
5.2308
5.2318
5.2327
5.2339
5.2349
5.2357
5.237
5.2379
5.2389
5.2401
5.2411
5.242
5.2432
5.2442
5.2451
5.2463
5.2472
5.2482
5.2494
5.2504
5.2513
5.2526
5.2534
5.2546
5.2555
5.2565
5.2577
5.2587
5.2596
5.2608
5.2617
5.2627
5.2639
5.2648
5.266
5.2669
5.2678
5.269
5.27
5.2709
5.2722
5.2731
5.274
5.2752
5.2761
5.2773
5.2783
5.2791
5.2804
5.2813
5.2822
5.2835
5.2844
5.2853
5.2866
5.2876
5.2885
5.2897
5.2907
5.2915
5.2928
5.2938
5.2947
5.2959
5.2969
5.2978
5.299
5.3
5.3009
5.3021
5.303
5.304
5.3052
5.306
5.3071
5.3083
5.3092
5.3103
5.3112
5.3123
5.3132
5.3144
5.3155
5.3164
5.3175
5.3184
5.3195
5.3204
5.3216
5.3227
5.3236
5.3248
5.3256
5.32

6.4073
6.4083
6.4095
6.4103
6.4115
6.4123
6.4135
6.4147
6.4155
6.4167
6.4178
6.4186
6.4198
6.4207
6.4218
6.4229
6.4238
6.425
6.4259
6.427
6.4281
6.429
6.4296
6.431
6.4321
6.4329
6.4341
6.435
6.4361
6.4373
6.4381
6.4392
6.4402
6.4413
6.4425
6.4433
6.4445
6.4453
6.4465
6.4476
6.4484
6.4496
6.4507
6.4517
6.4528
6.4537
6.4548
6.456
6.4568
6.458
6.4589
6.46
6.4611
6.462
6.4632
6.464
6.4652
6.4661
6.4672
6.4684
6.4692
6.4704
6.4712
6.4724
6.4733
6.4745
6.4753
6.4765
6.4776
6.4785
6.4797
6.4805
6.4817
6.4825
6.4837
6.4849
6.4857
6.4869
6.4878
6.4889
6.4898
6.4909
6.4921
6.493
6.4942
6.495
6.4962
6.497
6.4982
6.4993
6.5002
6.5013
6.5022
6.5034
6.5043
6.5054
6.5065
6.5075
6.5086
6.5094
6.5106
6.5115
6.5127
6.5138
6.5147
6.5159
6.5164
6.5179
6.519
6.5199
6.5211
6.5219
6.5231
6.524
6.5251
6.5262
6.5274
6.5283
6.5294
6.5305
6.5314
6.5326
6.5334
6.5346
6.5355
6.5367
6.5378
6.5387
6.5398
6.5407
6.5419
6.5427
6.5439
6.545
6.5459
6.5471
6.5479
6.5491
6.5499
6.5511
6.5523
6.5531
6.5543
6.5552
6.5563
6.

7.632
7.6328
7.634
7.6348
7.636
7.6371
7.638
7.6392
7.64
7.6411
7.6423
7.6431
7.6443
7.6451
7.6463
7.6475
7.6482
7.6494
7.6503
7.6514
7.6526
7.6534
7.6545
7.6557
7.6566
7.6578
7.6586
7.6598
7.661
7.6618
7.6629
7.6639
7.6649
7.6661
7.6669
7.6681
7.6693
7.6701
7.6713
7.6721
7.6733
7.6744
7.6752
7.6764
7.6776
7.6787
7.6795
7.6807
7.6816
7.6827
7.6838
7.6847
7.6858
7.6867
7.6879
7.689
7.6899
7.691
7.6919
7.693
7.6939
7.6951
7.6962
7.6972
7.6983
7.6992
7.7003
7.7012
7.7023
7.7032
7.7044
7.7055
7.7064
7.7075
7.7084
7.7095
7.7104
7.7116
7.7127
7.7136
7.7148
7.7156
7.7162
7.7176
7.7188
7.7197
7.7205
7.7217
7.7225
7.7236
7.7245
7.7256
7.7268
7.7279
7.7287
7.7296
7.7307
7.7316
7.7327
7.7339
7.7347
7.7359
7.7368
7.7379
7.7388
7.74
7.7411
7.742
7.7431
7.7439
7.7451
7.746
7.7471
7.7483
7.7491
7.7503
7.7512
7.7523
7.7532
7.7543
7.7555
7.7564
7.7575
7.7584
7.7595
7.7604
7.7612
7.7624
7.7635
7.7644
7.7656
7.7667
7.7676
7.7688
7.7696
7.7708
7.7716
7.7727
7.7736
7.7748
7.776
7.7768
7.7779
7.7788
7.78
7.

# Converts Dataset Results to CSV

In [12]:
dataset = datasaver.dataset
#Converts to pandas dataframe
df = dataset.to_pandas_dataframe()
#Converts dataframe to csv
df.to_csv(csvPath)

# Plot Dataset Results

In [ ]:
plot_dataset(dataset)